# Shakespeare

## Data Download

### Setup Instructions

#### Generated federated dataset

Run `bash preprocess.sh` with a choice of the following tags:
- `-s` := 'iid' to sample in an i.i.d. manner, or 'niid' to sample in a non-i.i.d. manner; more information on i.i.d. versus non-i.i.d. is included in the 'Notes' section
- `--iu` := number of users, if i.i.d. sampling; expressed as a fraction of the total number of users; default is 0.01
- `--sf` := fraction of data to sample, written as a decimal; default is 0.1
- `-k` := minimum number of samples per user
- `-t` := 'user' to partition users into train-test groups, or 'sample' to partition each user's samples into train-test groups; default is 'sample'
- `--tf` := fraction of data in training set, written as a decimal; default is 0.9
- `--raw` := include users' raw text data in all_data.json
- `--smplseed` := seed to be used before random sampling of data
- `--spltseed` :=  seed to be used before random split of data

**Small-sized Dataset** (Optional)

'-tf 0.8' reflects the train-test split used in the [FedAvg paper](https://arxiv.org/pdf/1602.05629.pdf)

In [ ]:
# Clear tmp folder
!cd ../benchmark/datasets/shakespeare/ && rm -rf data/rem_user_data data/sampled_data data/test data/train

# Download data and sampling
!cd ../benchmark/datasets/shakespeare/ && bash preprocess.sh -s niid --sf 0.2 -k 0 -t sample -tf 0.8

**Full-sized Dataset** (Optional)

In [ ]:
# Clear tmp folder
!cd ../benchmark/datasets/shakespeare/ && rm -rf data/rem_user_data data/sampled_data data/test data/train

# Download data and sampling
!cd ../benchmark/datasets/shakespeare/ && bash preprocess.sh -s niid --sf 1.0 -k 0 -t sample -tf 0.8

### Notes

- More details on i.i.d. versus non-i.i.d.:
  - In the i.i.d. sampling scenario, each data-point is equally likely to be sampled. Thus, all users have the same underlying distribution of data.
  - In the non-i.i.d. sampling scenario, the underlying distribution of data for each user is consistent with the raw data. Since we assume that data distributions vary between user in the raw data, we refer to this sampling process as non-i.i.d.
- More details on `preprocess.sh`:
  - The order in which `preprocess.sh` processes data is 1. generating all_data, 2. sampling, 3. removing users, and 4. creating train-test split. The script will look at the data in the last generated directory and continue preprocessing from that point. For example, if the `all_data` directory has already been generated and the user decides to skip sampling and only remove users with the `-k` tag (i.e. running `preprocess.sh -k 50`), the script will effectively apply a remove user filter to data in `all_data` and place the resulting data in the `rem_user_data` directory.
  - File names provide information about the preprocessing steps taken to generate them. For example, the `all_data_niid_1_keep_64.json` file was generated by first sampling 10 percent (.1) of the data `all_data.json` in a non-i.i.d. manner and then applying the `-k 64` argument to the resulting data.
- Each .json file is an object with 4 keys:
  1. 'users', a list of users
  2. 'hierarchies', a list of strings, with each string representing the group that the respective user belongs in; not present in i.i.d. data
  3. 'num_samples', a list of the number of samples for each user, and
  4. 'user_data', an object with user names as keys; the values are represented as objects with keys 'x', 'y', and 'raw'. 'x' and 'y' refer to strings and their corresponding next character. 'raw' refers to the text data from which the model data was extracted; this key appears only in all_data.json, and only when the '--raw' tag is used.
- Run `./stats.sh` to get statistics of data (data/all_data/all_data.json must have been generated already)
- In order to run reference implementations, the `-t sample` tag must be used when running `./preprocess.sh`

### Valid dataset

In [2]:
from benchmark.datasets.shakespeare import get_shakespeare
dataset = get_shakespeare('../benchmark/datasets/shakespeare/data')
print(dataset)
x, y = dataset[0]
print(x.shape, y.shape)
print(f'vocab size: {dataset.vocab_size}')

Shakespeare(total_parts: 660, total_samples: 3678451, current_parts: 0)
torch.Size([80]) torch.Size([80])
vocab size: 80


## FedAvg, FedSGD, FedEla, FedProx, FedScaffold

Run following commands in the root path of `benchmark-lightly`.

```bash
function cmd(){
    fed_optim=$1

    task_name="shakespeare"
    exp_name=${fed_optim}_${task_name}

    # Delete cache file
    rm -rf /tmp/${exp_name}.share
    rm -rf /tmp/${exp_name}
    rm -rf ./logs/${task_name}/${fed_optim}

    # Run
    python -m openfed.tools.launch --nproc_per_node 6  --logdir /tmp benchmark/run.py\
        --fed_init_method file:///tmp/${exp_name}.share\
        --task ${task_name}\
        --data_root benchmark/datasets/${task_name}/data\
        --epochs 1\
        --rounds 20\
        --act_clts 100\
        --tst_act_clts 100\
        --max_acg_step -1\
        --optim ${fed_optim}\
        --optim_args momentum:0.9 weight_decay:1e-4\
        --follower_lr 1e-1\
        --leader_lr 1.0\
        --bz 10\
        --gpu\
        --log_level SUCCESS\
        --log_dir logs\
        --exp_name ${exp_name}\
        --seed 0
}
```

### Run All

```bash
cmd 'fedavg'; cmd 'fedsgd'; cmd 'fedela'; cmd 'fedprox'; cmd 'fedscaffold'
```

## Plot Curves

In [ ]:
%matplotlib inline

from benchmark.utils.plot import plot

task_name = "shakespeare"

items = dict(
    FedAvg=f'../logs/{task_name}/fedavg_{task_name}/{task_name}.json',
    FedSgd=f'../logs/{task_name}/fedsgd_{task_name}/{task_name}.json',
    FedEla=f'../logs/{task_name}/fedela_{task_name}/{task_name}.json',
    FedProx=f'../logs/{task_name}/fedprox_{task_name}/{task_name}.json',
    FedScaffold=f'../logs/{task_name}/fedscaffold_{task_name}/{task_name}.json',
)

files = items.values()
labels = items.keys()

### Train Accuracy

In [ ]:
plot(
    files=files,
    labels=labels,
    attributes="accuracy",
    mode='train'
)

### Train Loss

In [ ]:
plot(
    files=files,
    labels=labels,
    attributes="loss",
    mode="train"
)

### Test Accuracy

In [ ]:
plot(
    files=files,
    labels=labels,
    attributes="accuracy",
    mode="test"
)

### Test Loss

In [ ]:
plot(
    files=files,
    labels=labels,
    attributes="loss",
    mode='test'
)